In [24]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from math import radians, sin, cos, sqrt, atan2
import plotly.figure_factory as ff
import json
import pytz
import folium

# Set random seed for reproducibility
np.random.seed(42)

In [4]:
# Haversine formula to calculate distance between two points on Earth
def haversine(lat1, lng1, lat2, lng2):
    R = 6371  # Earth's radius in km
    lat1, lng1, lat2, lng2 = map(radians, [lat1, lng1, lat2, lng2])
    dlat = lat2 - lat1
    dlng = lng2 - lng1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlng/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

In [148]:
# Set base time to current time in IST
base_time = datetime.now(pytz.timezone('Asia/Kolkata'))

# Generate riders data
riders = pd.DataFrame({
    'rider_id': [f'R{str(i).zfill(3)}' for i in range(1, 101)],
    'lat': np.random.uniform(18.48, 18.52, 100),  
    'lng': np.random.uniform(73.88, 73.92, 100), 
    'soc_pct': np.random.randint(20, 36, 100), 
    'status': np.random.choice(['idle', 'on_gig'], 100, p=[0.5, 0.5]),
    'km_to_finish': np.nan,
    'est_finish_ts': None
})

# Assign km_to_finish and est_finish_ts for on_gig riders
on_gig = riders['status'] == 'on_gig'
riders.loc[on_gig, 'km_to_finish'] = np.round(np.random.uniform(0.5, 4, on_gig.sum()), 1)  
riders.loc[on_gig, 'est_finish_ts'] = [
    (base_time + timedelta(minutes=int(minutes))).isoformat()
    for minutes in np.random.randint(5, 31, on_gig.sum()) 
]

# Fill NaN values
riders.fillna({'km_to_finish': 0, 'est_finish_ts': ''}, inplace=True)



In [149]:
riders.head()

,rider_id,lat,lng,soc_pct,status,km_to_finish,est_finish_ts
0,R001,18.488983,73.893023,27,on_gig,2.4,2025-05-19T18:07:47.131550+05:30
1,R002,18.500008,73.919344,28,idle,0.0,
2,R003,18.513618,73.906694,29,on_gig,3.6,2025-05-19T18:13:47.131550+05:30
3,R004,18.508052,73.881754,21,idle,0.0,
4,R005,18.511562,73.893065,35,idle,0.0,


In [150]:
# Generate stations data
stations = pd.DataFrame({
    'station_id': ['S_A', 'S_B', 'S_C'], 
    'lat': np.random.uniform(18.48, 18.52, 3),  
    'lng': np.random.uniform(73.88, 73.92, 3), 
    'queue_len': np.random.randint(0, 6, 3)  
})

In [151]:
stations.head()

,station_id,lat,lng,queue_len
0,S_A,18.488381,73.918753,2
1,S_B,18.485804,73.919676,4
2,S_C,18.516175,73.911590,1


In [152]:
# Save input data as JSON
riders.to_json('riders.json', orient='records', lines=True)
stations.to_json('stations.json', orient='records', lines=True)
print('Input data saved: riders.json, stations.json')

Input data saved: riders.json, stations.json


In [155]:
def optimize_swaps(riders, stations, base_time, speed_kmh=30, time_window_min=60, swap_duration_min=4, max_queue_length=5, soc_threshold=10, soc_consumption_rate=4):

    plan = []
    queue_timeline = {sid: [] for sid in stations['station_id']}
    total_detour = 0
    low_soc_riders = []
    unassigned_riders = []
    rider_details = []

    # Prioritize riders by low SOC
    riders['priority'] = riders['soc_pct']
    riders = riders.sort_values('priority', ascending=True)

    for _, rider in riders.iterrows():
        soc = rider['soc_pct']
        return_lat, return_lng = rider['lat'], rider['lng']

        if rider['status'] == 'on_gig':
            dist = rider['km_to_finish']
            soc = max(0, soc - dist * soc_consumption_rate)
            try:
                depart_time = datetime.fromisoformat(rider['est_finish_ts'])
            except ValueError:
                print(f"Invalid est_finish_ts for rider {rider['rider_id']}: {rider['est_finish_ts']}")
                unassigned_riders.append({
                    'rider_id': rider['rider_id'],
                    'reason': "Invalid est_finish_ts format"
                })
                continue
            start_lat, start_lng = rider['lat'], rider['lng']
        else:
            depart_time = base_time
            start_lat, start_lng = rider['lat'], rider['lng']

        # Compute distances to all stations
        distances = {}
        for _, station in stations.iterrows():
            dist = haversine((start_lat, start_lng), (station['lat'], station['lng']))
            distances[station['station_id']] = (dist, station['lat'], station['lng'])
        sorted_distances = sorted(distances.items(), key=lambda x: x[1][0])

        # Check if swap is needed
        nearest_dist, nearest_lat, nearest_lng = sorted_distances[0][1]
        nearest_station_id = sorted_distances[0][0]
        projected_soc = soc - nearest_dist * soc_consumption_rate
        needs_swap = 10 <= projected_soc <= 20 or soc <= soc_threshold

        # Debug: Why is rider skipped or considered?
        print(f"Rider {rider['rider_id']}: soc={soc:.2f}%, projected_soc={projected_soc:.2f}%, needs_swap={needs_swap}, nearest_dist={nearest_dist:.2f} km")

        if needs_swap:
            low_soc_riders.append({
                'rider_id': rider['rider_id'],
                'projected_soc': projected_soc,
                'station_id': nearest_station_id
            })

        if not needs_swap:
            continue

        # Find a station with available queue and sufficient SOC
        best_station = None
        best_timestamps = None
        detour_for_total = 0
        found_station = False
        unassigned_reason = "All stations have full queues, are too far, or SOC would drop below 8%"

        # First, try stations with available queue slots
        for station_id, (dist, station_lat, station_lng) in sorted_distances:
            # Relaxed SOC check: Allow SOC >= 8% at arrival
            arrival_soc = soc - dist * soc_consumption_rate
            if arrival_soc < 10:
                print(f"  Skip station {station_id} for {rider['rider_id']}: arrival_soc={arrival_soc:.2f}% < 10%")
                continue

            station = stations[stations['station_id'] == station_id].iloc[0]
            travel_time = (dist / speed_kmh) * 60
            arrive_time = depart_time + timedelta(minutes=travel_time)

            if arrive_time > base_time + timedelta(minutes=time_window_min):
                print(f"  Skip station {station_id} for {rider['rider_id']}: arrive_time={arrive_time} exceeds {time_window_min}-min window")
                continue

            queue = queue_timeline[station['station_id']]
            initial_queue = station['queue_len']
            scheduled_queue = sum(1 for s, e in queue if s <= arrive_time <= e)
            total_queue = initial_queue + scheduled_queue
            if total_queue >= max_queue_length:
                print(f"  Skip station {station_id} for {rider['rider_id']}: total_queue={total_queue} >= {max_queue_length}")
                continue

            queue_end = min((e for _, e in queue if e > arrive_time), default=arrive_time)
            swap_start = max(arrive_time, queue_end)
            swap_end = swap_start + timedelta(minutes=swap_duration_min)

            if swap_end > base_time + timedelta(minutes=time_window_min):
                print(f"  Skip station {station_id} for {rider['rider_id']}: swap_end={swap_end} exceeds {time_window_min}-min window")
                continue

            if rider['status'] == 'on_gig':
                original_dist = haversine((start_lat, start_lng), (return_lat, return_lng))
                detour = dist + haversine((station['lat'], station['lng']), (return_lat, return_lng)) - original_dist
            else:
                detour = dist + haversine((station['lat'], station['lng']), (return_lat, return_lng))

            best_station = station
            best_timestamps = {
                'depart_ts': depart_time.isoformat(),
                'arrive_ts': arrive_time.isoformat(),
                'swap_start_ts': swap_start.isoformat(),
                'swap_end_ts': swap_end.isoformat(),
                'eta_back_lat': return_lat,
                'eta_back_lng': return_lng
            }
            detour_for_total = detour
            found_station = True
            print(f"  Assigned {rider['rider_id']} to {station_id}: arrival_soc={arrival_soc:.2f}%, detour={detour:.2f} km")
            break

        # If no station is available, wait at the nearest station
        if not found_station:
            station_id, (dist, station_lat, station_lng) = sorted_distances[0]
            station = stations[stations['station_id'] == station_id].iloc[0]
            travel_time = (dist / speed_kmh) * 60
            arrive_time = depart_time + timedelta(minutes=travel_time)

            if arrive_time > base_time + timedelta(minutes=time_window_min):
                unassigned_riders.append({
                    'rider_id': rider['rider_id'],
                    'reason': f"Cannot reach nearest station {station_id} within {time_window_min}-minute window (arrive at {arrive_time})"
                })
                print(f"  Unassigned {rider['rider_id']}: Cannot reach {station_id} in time")
                continue

            arrival_soc = soc - dist * soc_consumption_rate
            if arrival_soc < 10:
                unassigned_riders.append({
                    'rider_id': rider['rider_id'],
                    'reason': f"Insufficient SOC to reach nearest station {station_id} (SOC would be {arrival_soc:.2f}%)"
                })
                print(f"  Unassigned {rider['rider_id']}: arrival_soc={arrival_soc:.2f}% < 10%")
                continue

            queue = queue_timeline[station['station_id']]
            initial_queue = station['queue_len']
            queue_ends = sorted([e for s, e in queue if e > arrive_time and e <= base_time + timedelta(minutes=time_window_min)])
            for slot in queue_ends:
                swap_start = slot
                swap_end = swap_start + timedelta(minutes=swap_duration_min)
                if swap_end <= base_time + timedelta(minutes=time_window_min):
                    scheduled_queue = sum(1 for s, e in queue if s <= swap_start <= e)
                    total_queue = initial_queue + scheduled_queue
                    if total_queue < max_queue_length:
                        if rider['status'] == 'on_gig':
                            original_dist = haversine((start_lat, start_lng), (return_lat, return_lng))
                            detour = dist + haversine((station['lat'], station['lng']), (return_lat, return_lng)) - original_dist
                        else:
                            detour = dist + haversine((station['lat'], station['lng']), (return_lat, return_lng))

                        best_station = station
                        best_timestamps = {
                            'depart_ts': depart_time.isoformat(),
                            'arrive_ts': arrive_time.isoformat(),
                            'swap_start_ts': swap_start.isoformat(),
                            'swap_end_ts': swap_end.isoformat(),
                            'eta_back_lat': return_lat,
                            'eta_back_lng': return_lng
                        }
                        detour_for_total = detour
                        found_station = True
                        print(f"  Assigned {rider['rider_id']} to {station_id} (wait): arrival_soc={arrival_soc:.2f}%, detour={detour:.2f} km")
                        break

        if found_station and best_station is not None:
            plan.append({
                'rider_id': rider['rider_id'],
                'station_id': best_station['station_id'],
                'depart_ts': best_timestamps['depart_ts'],
                'arrive_ts': best_timestamps['arrive_ts'],
                'swap_start_ts': best_timestamps['swap_start_ts'],
                'swap_end_ts': best_timestamps['swap_end_ts'],
                'eta_back_lat': best_timestamps['eta_back_lat'],
                'eta_back_lng': best_timestamps['eta_back_lng']
            })
            queue_timeline[best_station['station_id']].append((swap_start, swap_end))
            total_detour += detour_for_total
            rider_details.append({
                'rider_id': rider['rider_id'],
                'projected_soc': projected_soc,
                'nearest_station': nearest_station_id,
                'assigned_station': best_station['station_id'],
                'detour_km': detour_for_total
            })
        else:
            unassigned_riders.append({
                'rider_id': rider['rider_id'],
                'reason': unassigned_reason
            })
            print(f"  Unassigned {rider['rider_id']}: {unassigned_reason}")

    # Output
    print("\n=== Battery Swap Summary ===")
    if rider_details:
        print("\nRiders Needing Battery Swaps:")
        print("-------------------------------------------------------")
        print("Rider ID | Projected SOC | Nearest Station | Assigned Station | Detour (km)")
        print("-------------------------------------------------------")
        for detail in rider_details:
            print(f"{detail['rider_id']:8} | {detail['projected_soc']:12.2f}% | {detail['nearest_station']:14} | {detail['assigned_station']:15} | {detail['detour_km']:10.2f}")
        print("-------------------------------------------------------")
    else:
        print("\nNo riders needed battery swaps.")

    print("\nStation Usage Summary:")
    print("---------------------------------")
    print("Station ID | Initial Queue | Assigned Riders | Total Queue")
    print("---------------------------------")
    for station_id in stations['station_id']:
        initial_queue = stations[stations['station_id'] == station_id]['queue_len'].iloc[0]
        assigned_riders = len(queue_timeline[station_id])
        total_queue = initial_queue + assigned_riders
        print(f"{station_id:9} | {initial_queue:12} | {assigned_riders:14} | {total_queue:10}")
    print("---------------------------------")

    if unassigned_riders:
        print('\nRiders who could not be assigned:')
        for rider in unassigned_riders:
            print(f"Rider {rider['rider_id']}: {rider['reason']}")
    else:
        print('\nAll riders needing swaps were assigned to stations.')
    print(f'Total detour kilometers: {total_detour:.2f}')

    # Return DataFrame with consistent columns
    if not plan:
        plan = pd.DataFrame(columns=['rider_id', 'station_id', 'depart_ts', 'arrive_ts', 'swap_start_ts', 'swap_end_ts', 'eta_back_lat', 'eta_back_lng'])
    else:
        plan = pd.DataFrame(plan)
    return plan, low_soc_riders, total_detour

In [156]:
plan, low_soc_riders, total_detour = optimize_swaps(
    riders, stations, base_time,
    speed_kmh=30, time_window_min=60, swap_duration_min=4,
    max_queue_length=5, soc_threshold=10, soc_consumption_rate=4
)
plan.to_json('plan_output.json', orient='records', lines=True)
print('Plan output saved: plan_output.json')

Rider R045: soc=5.60%, projected_soc=-0.20%, needs_swap=True, nearest_dist=1.45 km
  Skip station S_C for R045: arrival_soc=-0.20% < 10%
  Skip station S_A for R045: arrival_soc=-1.97% < 10%
  Skip station S_B for R045: arrival_soc=-3.17% < 10%
  Unassigned R045: arrival_soc=-0.20% < 10%
Rider R035: soc=20.00%, projected_soc=17.51%, needs_swap=True, nearest_dist=0.62 km
  Assigned R035 to S_C: arrival_soc=17.51%, detour=1.25 km
Rider R081: soc=20.00%, projected_soc=10.68%, needs_swap=True, nearest_dist=2.33 km
  Assigned R081 to S_C: arrival_soc=10.68%, detour=4.66 km
Rider R067: soc=20.00%, projected_soc=12.30%, needs_swap=True, nearest_dist=1.93 km
  Assigned R067 to S_C: arrival_soc=12.30%, detour=3.85 km
Rider R052: soc=15.80%, projected_soc=7.44%, needs_swap=False, nearest_dist=2.09 km
Rider R033: soc=21.00%, projected_soc=10.76%, needs_swap=True, nearest_dist=2.56 km
  Assigned R033 to S_C: arrival_soc=10.76%, detour=5.12 km
Rider R004: soc=21.00%, projected_soc=7.91%, needs_swap

In [157]:
plan.head()

,rider_id,station_id,depart_ts,arrive_ts,swap_start_ts,swap_end_ts,eta_back_lat,eta_back_lng
0,R035,S_C,2025-05-19T18:02:47.131550+05:30,2025-05-19T18:04:01.966840+05:30,2025-05-19T18:04:01.966840+05:30,2025-05-19T18:08:01.966840+05:30,18.516795,73.905712
1,R081,S_C,2025-05-19T18:02:47.131550+05:30,2025-05-19T18:07:26.855707+05:30,2025-05-19T18:08:01.966840+05:30,2025-05-19T18:12:01.966840+05:30,18.516077,73.889483
2,R067,S_C,2025-05-19T18:02:47.131550+05:30,2025-05-19T18:06:38.143288+05:30,2025-05-19T18:08:01.966840+05:30,2025-05-19T18:12:01.966840+05:30,18.519897,73.893759
3,R033,S_C,2025-05-19T18:02:47.131550+05:30,2025-05-19T18:07:54.337938+05:30,2025-05-19T18:08:01.966840+05:30,2025-05-19T18:12:01.966840+05:30,18.500012,73.894301
4,R028,S_C,2025-05-19T18:02:47.131550+05:30,2025-05-19T18:04:17.002908+05:30,2025-05-19T18:08:01.966840+05:30,2025-05-19T18:12:01.966840+05:30,18.512424,73.905691


In [158]:
if not plan.empty:
    m = folium.Map(location=[18.45, 73.55], zoom_start=10, tiles='OpenStreetMap')
    
    for _, rider in riders.iterrows():
        is_low_soc = any(r['rider_id'] == rider['rider_id'] for r in low_soc_riders)
        color = 'red' if is_low_soc else 'blue'
        folium.Marker(
            [rider['lat'], rider['lng']],
            popup=f"Rider {rider['rider_id']}<br>SOC: {rider['soc_pct']:.2f}%<br>Status: {rider['status']}",
            icon=folium.Icon(color=color)
        ).add_to(m)
    
    queue_timeline = {sid: [] for sid in stations['station_id']}
    for _, swap in plan.iterrows():
        swap_start = datetime.fromisoformat(swap['swap_start_ts'])
        swap_end = datetime.fromisoformat(swap['swap_end_ts'])
        queue_timeline[swap['station_id']].append((swap_start, swap_end))
    for _, station in stations.iterrows():
        initial_queue = station['queue_len']
        assigned_riders = len(queue_timeline[station['station_id']])
        total_queue = initial_queue + assigned_riders
        folium.Marker(
            [station['lat'], station['lng']],
            popup=f"Station {station['station_id']}<br>Initial Queue: {initial_queue}<br>Total Queue: {total_queue}",
            icon=folium.Icon(color='green')
        ).add_to(m)
    
    for _, swap in plan.iterrows():
        rider = riders[riders['rider_id'] == swap['rider_id']].iloc[0]
        station = stations[stations['station_id'] == swap['station_id']].iloc[0]
        folium.PolyLine(
            [[rider['lat'], rider['lng']], [station['lat'], station['lng']]],
            color='purple', weight=2, opacity=0.5,
            popup=f"{swap['rider_id']} to {swap['station_id']}<br>Swap: {swap['swap_start_ts']} to {swap['swap_end_ts']}"
        ).add_to(m)
    
    m.save('map_visualization.html')
    print('Pune map saved: map_visualization.html')
else:
    print('No swaps scheduled; no map generated.')

Pune map saved: map_visualization.html
